In [1]:
import numpy as np
from lipton_tarjan import PlanarGraph, PlanarGraphEdges, PlanarSeparator, \
        PlanarGraphConstructor, PlanarGraphGenerator
from planar_ising.expanded_dual_graph_constructor import construct
from planar_ising.partial_spin_configuration import PartialSpinConfiguration
from planar_ising import WilsonSampling, PlanarIsingModelGenerator, IsingInferenceAndSampling
from planar_ising.wilson_sampling import _collapse_perfect_matching

In [ ]:
g = PlanarGraphConstructor.construct_from_ordered_adjacencies(
    [[1, 3, 2], [0, 2, 4], [1, 0, 5], [0, 4, 5], [3, 1, 5], [2, 3, 4]]
)

In [6]:
def output_graph(graph):

    print('Adjacencies:')
    for v in range(graph.size):
        print('\t{0} -> {1}'.format(v, list(zip(graph.get_adjacent_vertices(v),
                                                graph.get_incident_edge_indices(v)))))

    print('Edges:')
    for e in range(graph.edges_count):
        print('\t{0} -> ({1}, {2})'.format(e, graph.edges.vertex1[e], graph.edges.vertex2[e]))

In [ ]:
output_graph(g)

In [ ]:
pmm = np.array([False]*9)
pmm[1] = True
pmm[3] = True
pmm[8] = True

In [ ]:
nvm, neim, gg = _collapse_perfect_matching(g, pmm)

print(nvm)
print(neim)
output_graph(gg)

In [7]:
g = PlanarGraphConstructor.construct_from_ordered_adjacencies([
    [1, 9, 2],
    [0, 2, 3],
    [0, 6, 1],
    [1, 5, 4],
    [3, 5, 10],
    [3, 8, 4],
    [2, 7, 8],
    [6, 11, 8],
    [5, 6, 7],
    [0, 10, 11],
    [4, 11, 9],
    [7, 9, 10]
])

In [8]:
output_graph(g)

Adjacencies:
	0 -> [(1, 0), (9, 1), (2, 2)]
	1 -> [(0, 0), (2, 3), (3, 4)]
	2 -> [(0, 2), (6, 5), (1, 3)]
	3 -> [(1, 4), (5, 6), (4, 7)]
	4 -> [(3, 7), (5, 8), (10, 9)]
	5 -> [(3, 6), (8, 10), (4, 8)]
	6 -> [(2, 5), (7, 11), (8, 12)]
	7 -> [(6, 11), (11, 13), (8, 14)]
	8 -> [(5, 10), (6, 12), (7, 14)]
	9 -> [(0, 1), (10, 15), (11, 16)]
	10 -> [(4, 9), (11, 17), (9, 15)]
	11 -> [(7, 13), (9, 16), (10, 17)]
Edges:
	0 -> (0, 1)
	1 -> (0, 9)
	2 -> (0, 2)
	3 -> (1, 2)
	4 -> (1, 3)
	5 -> (2, 6)
	6 -> (3, 5)
	7 -> (3, 4)
	8 -> (4, 5)
	9 -> (4, 10)
	10 -> (5, 8)
	11 -> (6, 7)
	12 -> (6, 8)
	13 -> (7, 11)
	14 -> (7, 8)
	15 -> (9, 10)
	16 -> (9, 11)
	17 -> (10, 11)


In [9]:
pmm = np.array([False]*18)
pmm[2] = True
pmm[4] = True
pmm[8] = True
pmm[12] = True
pmm[13] = True
pmm[15] = True

In [10]:
nvm, neim, gg = _collapse_perfect_matching(g, pmm)

print(nvm)
print(neim)
output_graph(gg)

[0 1 0 1 2 2 3 4 3 5 5 4]
[ 0  1 -1  0 -1  2  3  3 -1  4  5  6 -1 -1  6 -1  7  7]
Adjacencies:
	0 -> [(3, 2), (1, 0), (5, 1)]
	1 -> [(2, 3), (0, 0)]
	2 -> [(3, 5), (5, 4), (1, 3)]
	3 -> [(4, 6), (2, 5), (0, 2)]
	4 -> [(5, 7), (3, 6)]
	5 -> [(4, 7), (0, 1), (2, 4)]
Edges:
	0 -> (0, 1)
	1 -> (0, 5)
	2 -> (0, 3)
	3 -> (1, 2)
	4 -> (2, 5)
	5 -> (2, 3)
	6 -> (3, 4)
	7 -> (4, 5)


In [ ]:
'''
for size in [10, 100, 1000]:
    for i in range(100):

        g = PlanarGraphGenerator.generate_random_graph(100, 0.8)

        gem, edg = construct(g)

        pmm = (gem.second == -1)

        _collapse_perfect_matching(edg, pmm);
''';

In [2]:
def test_partial_spin_configuration(size):

    partial_spin_configuration = PartialSpinConfiguration(size)

    components = np.arange(size)

    rules = []
    
    for rule_index in range(size - 1):

        vertex1 = np.random.choice(size)
        component1 = components[vertex1]

        component_size = (components == component1).sum()

        vertex2_index = np.random.choice(size - component_size)

        vertex2 = 0

        for vertex2 in range(size):

            if components[vertex2] == component1:
                continue

            if vertex2_index == 0:
                break

            vertex2_index -= 1

        component2 = components[vertex2]
                
        for vertex in range(size):
            if components[vertex] == component2:
                components[vertex] = component1

        set_equal_values = np.random.choice([True, False])

        rules.append((vertex1, vertex2, set_equal_values))
        
        partial_spin_configuration.set_spin_pair(vertex1, vertex2, set_equal_values)

        '''
        print(vertex1, vertex2, set_equal_values)
        print(partial_spin_configuration.spin_values)
        print(partial_spin_configuration._component_indices)
        print(partial_spin_configuration._next_vertices_in_components)
        print(partial_spin_configuration._component_first_vertices)
        '''

    #print('CHECK')

    for vertex1, vertex2, set_equal_values in rules:

        spin_value1 = partial_spin_configuration.spin_values[vertex1]
        spin_value2 = partial_spin_configuration.spin_values[vertex2]

        if (spin_value1 == spin_value2) != set_equal_values:
            print(vertex1, vertex2, set_equal_values)
            print(partial_spin_configuration.spin_values)
            return False

    return True

In [15]:
for size in [10, 100, 1000]:
    for i in range(100):
        if not test_partial_spin_configuration(size):
            print('SHIEET')

In [8]:
g = PlanarGraphConstructor.construct_from_ordered_adjacencies([
    [1], [0]
])

In [9]:
PlanarSeparator.mark_separation(g)

array([<SeparationClass.SEPARATOR: 2>, <SeparationClass.FIRST_PART: 0>], dtype=object)

In [3]:
import time

In [8]:
s = IsingInferenceAndSampling(m)
start = time.time()
s.prepare_for_sampling()
print(time.time() - start)
start = time.time()
s.sample_spin_configurations(1)
print(time.time() - start)

27.859748601913452
36.35558319091797


In [7]:
m = PlanarIsingModelGenerator.generate_random_model(10000, 0.8, 0.1)

ws = WilsonSampling(m)

start = time.time()
ws.prepare_for_sampling()
print(time.time() - start)
start = time.time()
ws.sample_spin_configurations(1)
print(time.time() - start)

20.074562549591064
55.340888261795044
